# Shaikh's real effective exchange rate
### Calculations on the real effectice exchange rate between japan and the US 

The goal is to reproduce figure 11.4 and 11.5 from the book "Capitalism - Competition, Conflict, Crises" 

The real effective exchange rate is calculated as the vertically integrated unit labor costs (vulc) times a factor of the share of exports (tau).

Theta can have the form of the share of exports over GDP (exp/gdp) or of consumer price index over producer price index (CPI/PPI)

Therefore: 

$ \frac{vulcr}{vulcr*} \cdot \frac{\tau}{\tau*}$

or equivalent

$ \frac{(\frac{ULC}{CPI})}{(\frac{ULC*}{CPI*})} \cdot \frac{(\frac{exp}{gdp})}{(\frac{exp*}{gdp*})}$

# import data

### Japan

In [1]:
import pandas as pd

In [2]:
exp_gdp = pd.read_csv("EXP-GDP/API_NE.EXP.GNFS.ZS_DS2_en_csv_v2_431603.csv",skiprows=3,delimiter=",")
exp_gdp.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,Unnamed: 68
0,Aruba,ABW,Exports of goods and services (% of GDP),NE.EXP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,72.852913,71.820435,72.548690,72.252774,73.234592,55.32681,69.069709,83.124605,NaN,NaN
1,Africa Eastern and Southern,AFE,Exports of goods and services (% of GDP),NE.EXP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,24.046024,23.249929,23.824897,25.315921,23.903124,22.03616,25.922406,27.595354,24.50316,NaN


In [3]:
# % share of Exports over GDP on a YEARLY basis 
print(exp_gdp[exp_gdp["Country Code"] == "JPN"].columns)

exp_gdp_ss = exp_gdp[exp_gdp["Country Code"].isin(["JPN","USA"]) ].drop(
    ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code','Unnamed: 68'],axis=1).T.rename(
    columns={119:'jpn_exp_gdp',251:'usa_exp_gdp'})
print(len(exp_gdp_ss))
exp_gdp_ss.head(2)

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       '2023', 'Unnamed: 68'],
      dtype='object')
64


,jpn_exp_gdp,usa_exp_gdp
1960,NaN,NaN
1961,NaN,NaN


In [4]:
# DAILY basis
usd_yen = pd.read_csv("FX-Rate-USD-YEN/dollar-yen-exchange-rate-historical-chart.csv",skiprows=15)
print(len(usd_yen))
print(usd_yen.head(2))

13784
         date   value
0  1971-01-04  357.73
1  1971-01-05  357.81


In [5]:
usd_yen["month"] = pd.to_datetime(usd_yen.date).dt.month
usd_yen["year"] = pd.to_datetime(usd_yen.date).dt.year
usd_yen.head(2)

,date,value,month,year
0,1971-01-04,357.73,1,1971
1,1971-01-05,357.81,1,1971


In [6]:
usd_yen_yearly = usd_yen.groupby("year").mean("value")
print(len(usd_yen_yearly))
print(usd_yen_yearly.head(2))
print(usd_yen_yearly.tail(2))

54
          value     month
year                     
1971  347.78569  6.456067
1972  303.12498  6.438247
           value     month
year                      
2023  140.734001  6.677019
2024  150.839595  2.765432


In [7]:
# CPI from the world bank on a YEARLY basis

jpn_cpi = pd.read_csv("JPN-CPI/API_FP.CPI.TOTL_DS2_en_csv_v2_527528.csv",skiprows=3,delimiter=",")

# _wb = WORLD BANK

cpi_wb_ss = exp_gdp[exp_gdp["Country Code"].isin(["JPN","USA"]) ].drop(
    ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code','Unnamed: 68'],axis=1).T.rename(
    columns={119:'jpn_cpi_wb',251:'usa_cpi_wb'})

print(len(cpi_wb_ss))
print(cpi_wb_ss.head(2))
print(cpi_wb_ss.tail(2))

64
      jpn_cpi_wb  usa_cpi_wb
1960         NaN         NaN
1961         NaN         NaN
      jpn_cpi_wb  usa_cpi_wb
2022   21.541723   11.633909
2023         NaN         NaN


In [8]:
# finished goods PPI from FRED  on a MONTHLY basis
jpn_ppi = pd.read_csv("JPN-PPI/PISPFG01JPM661N.csv")
jpn_ppi = jpn_ppi.rename(columns={'PISPFG01JPM661N':'jpn_ppi'})
print(len(jpn_ppi))
print(jpn_ppi.head(2))
print(jpn_ppi.tail(2))

748
         DATE  jpn_ppi
0  1960-01-01  62.8903
1  1960-02-01  62.8903
           DATE  jpn_ppi
746  2022-03-01     99.9
747  2022-04-01    100.7


In [9]:
jpn_ppi["jpn_ppi_year"] = pd.to_datetime(jpn_ppi.DATE).dt.year
jpn_ppi["jpn_ppi_month"] = pd.to_datetime(jpn_ppi.DATE).dt.month

In [10]:
# UNIT LABOR COSTS FROM FRED  // INDEX = 2015
jpn_ulc = pd.read_csv("JPN-ULC/ULQECU01JPQ661S.csv")
jpn_ulc = jpn_ulc.rename(columns={'ULQECU01JPQ661S':'jpn_ulc'})
print(len(jpn_ulc))
print(jpn_ulc.head(2))
print(jpn_ulc.tail(2))

175
         DATE    jpn_ulc
0  1980-01-01  67.467467
1  1980-04-01  68.412182
           DATE     jpn_ulc
173  2023-04-01  107.630371
174  2023-07-01  107.373853


In [11]:
jpn_ulc["jpn_ulc_year"] = pd.to_datetime(jpn_ulc.DATE).dt.year
jpn_ulc["jpn_ulc_month"] = pd.to_datetime(jpn_ulc.DATE).dt.month

In [21]:
jpn_ulc_year = jpn_ulc.groupby("jpn_ulc_year").mean("jpn_ulc")
print(jpn_ulc_year.shape)
jpn_ulc_year.head(2)

(44, 2)


,jpn_ulc,jpn_ulc_month
jpn_ulc_year,,
1980,69.224826,5.5
1981,73.788575,5.5


In [23]:
jpn_ulc_year.index

Index([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991,
       1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023],
      dtype='int32', name='jpn_ulc_year')

In [24]:
jpn_ulc_year[jpn_ulc_year.index > 1990]

,jpn_ulc,jpn_ulc_month
jpn_ulc_year,,
1991,104.333478,5.5
1992,105.473682,5.5
1993,106.147567,5.5
1994,108.528704,5.5
1995,109.837066,5.5
1996,110.555523,5.5
1997,112.481711,5.5
1998,111.411768,5.5
1999,109.961185,5.5


### United States

### US CPI
US CPI Time series selection from the file "cu.series". Basically it lists all the series  codes which time series does  mean what.

the following series is the most appropirate since it is us city average:

CUSR0000SA0      	0000	SA0	S	R	S	1982-84=100	All items in U.S. city average, all urban consumers, seasonally adjusted		1947	M01	2024	M05

In [12]:
us_cpi = pd.read_table("US-CPI/cu.data.1.AllItems",sep="\s+",engine='python')
print(len(us_cpi))
us_cpi.head(2)

61705


,series_id,year,period,value,footnote_codes
0,CUSR0000SA0,1947,M01,21.48,NaN
1,CUSR0000SA0,1947,M02,21.62,NaN


In [13]:
# avoided the wthiespace problem with an "\s+" import, meaning that all white spaces are explicitly used, not the first row only
print(us_cpi.columns)
#us_cpi_colnames = [colname.strip() for colname in list(us_cpi.columns.values)]
#us_cpi_colnames

Index(['series_id', 'year', 'period', 'value', 'footnote_codes'], dtype='object')


In [14]:
us_cpi_ss = us_cpi[us_cpi['series_id'] == 'CUSR0000SA0']
us_cpi_ss["
print(len(us_cpi_ss))

SyntaxError: unterminated string literal (detected at line 2) (3190709430.py, line 2)

In [ ]:
us_cpi_ss_year = us_cpi_ss.groupby("year").mean("value")
print(len(us_cpi_ss_year))
print(us_cpi_ss_year.head(2))
print(us_cpi_ss_year.tail(2))

### US Producer Price Index 
the producer price index for the us does not have a total aggregate, however, the two series that fit the closest are the following: 

PCUOMFG--OMFG--               	OMFG--	OMFG--	U	198412	PPI industry group data for Total manufacturing industries, not seasonally adjusted		1984	M12	2024	M05

PCUOMIN--OMIN--               	OMIN--	OMIN--	U	198412	PPI industry group data for Total mining industries, not seasonally adjusted		1984	M12	2024	M05

In [ ]:
us_ppi = pd.read_table("US-PPI/pc.data.01.aggregates",sep="\s+",engine='python')
print(us_ppi.columns)
print(us_ppi.iloc[3495,0],type(us_ppi.iloc[3495,0]))
us_ppi.tail(2)

In [ ]:
us_ppi_ss = us_ppi[(us_ppi["period"]=="M13") & (us_ppi["series_id"]=="PCUOMFG--OMFG--")]
print(len(us_ppi_ss))
print(us_ppi_ss.head(2))
print(us_ppi_ss.tail(2))


In [ ]:
us_ppi_ss["mining"] = list(us_ppi[(us_ppi["period"]=="M13") & (us_ppi["series_id"]=="PCUOMIN--OMIN--")]["value"].values)

In [ ]:
us_ppi_ss[["value","mining"]].plot()

### Unit Labor Costs

for unit labor costs the following note is an example: 

PRS 3100 6 11 1 = PRs -3100 - 06 - 11 - 1

Series - Sector -Class Code - Mesaure Code - Time 

Selection for "Sector": 
 - 3000	Manufacturing	0	T	4
 - 3100	Manufacturing, Durable Goods	1	T	5
 - 3200	Manufacturing, Nondurable Goods	1	T	6
 - 8400	Business	0	T	2
 - 8500	Nonfarm Business	0	T	1
 - 8800	Nonfinancial Corporations	0	T	3

Selection for "Time"
 - 1	% Change same quarter 1 year ago	0	T	1
 - 2	% Change from previous quarter	0	T	2
 - 3	Index (2017=100)	0	T	3


Therefore: 
 - Sector: 8800
 - ClassCode: 3
 - Measure Code: 11
 - Time: 3 (index)

Therefore: PRS+8800 3 11 3

Sidenote:Q05	AN AV	Annual Average

In [ ]:
us_ulc = pd.read_table("US-ULC/pr.data.1.AllData",sep="\s+",engine='python')
us_ulc.head(2)

In [ ]:
us_ulc_ss = us_ulc[ (us_ulc["series_id"] == "PRS88003113" ) & (us_ulc["period"] == "Q05") ]
print(len(us_ulc_ss))
print(us_ulc_ss.head(2))
print(us_ulc_ss.tail(2))